## Housekeeping

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, classification_report, mean_squared_error
import math


## Preamble - Data Cleaning

To clean our data, we have reused some of the code we had employed earlier on in the course. We first concatenate these files in the below cell block, and run this file through our scrubber which is where we handle our data cleaning. For further details, you can review the scrubber notebook.

In [ ]:
#Combining the two datasets and saving as a new csv
df1 = pd.read_csv('covid19-cdc-22207000.csv', keep_default_na=True, delimiter=',', skipinitialspace=True)
df2 = pd.read_csv('covid19-cdc-12468998.csv', keep_default_na=True, delimiter=',', skipinitialspace=True)

combined_df = pd.concat([df1, df2])

combined_df.to_csv('covid19-cdc-combined-22207000-12468998.csv', index=False)

print(len(combined_df))

The product of this scrubber is <b>finalised-covid19-cdc-combined-22207000-12468998.csv</b>. We will create our train/test dataframes from this csv file.

In [ ]:
#Reading the post-scrubbed CSV and checking length
df = pd.read_csv('finalised-covid19-cdc-combined-22207000-12468998.csv', keep_default_na=True, delimiter=',', skipinitialspace=True)

print(len(df))

# Task 1

# 1.1 Train/Test Split

With the data cleaning taken care of, we can now conduct a train text split on the data to prepare it for future machine learning tasks. Once the data is split, we can plot the train frame and investigate the relationships between the features.

In [ ]:
#data for task 1
df = pd.read_csv('finalised-covid19-cdc-combined-22207000-12468998.csv', keep_default_na=True, delimiter=',', skipinitialspace=True)

#train test split on data
train, test = train_test_split(df, test_size = 0.3, random_state=42)

In [ ]:
df.duplicated().sum()

In [ ]:
train.shape

In [ ]:
test.shape

# 1.2 Mapping our Training Set

## Correlation Between Continuous Features

In [ ]:
fig, axs = plt.subplots(figsize=(20, 6), ncols=1, nrows=1)
plt.scatter(train['case_positive_specimen_interval'], train['case_onset_interval'])
corr = train['case_onset_interval'].corr(train['case_positive_specimen_interval'])
print('Correlation: ', corr)
print('Weak positive correlation between these two variables.')
plt.show()

## Continuous Features Plotted Against Death

In [ ]:
fig, axs = plt.subplots(figsize=(20, 6), ncols=1, nrows=1)
plt.scatter(train['death_yn'], train['case_positive_specimen_interval'])
plt.show()

In [ ]:
fig, axs = plt.subplots(figsize=(20, 6), ncols=1, nrows=1)
plt.scatter(train['death_yn'], train['case_onset_interval'])
plt.show()

## Health Related Categorical Features Plotted Against Death

Here we are looking discern patterns from our dataset by plotting deaths (yes/no) against a select number of categorical features. The features we are going to examine first are those that relate to the health profile of our patients:

1. When the patient contracted COVID.
2. What US state the patient was in when they were reported of having COVID.
3. What age group the patient belongs to.
4. Whether the patient knew if they were exposed to COVID.
5. Whether or not the patient was hospitalised or not.
6. Whether or not the patient was taken to ICU.
7. And whether or not the patient exhibited underlying conditions which could exacerbate COVID.

In [ ]:
target_feature = train['death_yn']
select_features = [train['case_month'], train['res_state'], train['age_group'], train['hosp_yn'], train['exposure_yn'], train['icu_yn'], train['underlying_conditions_yn']]

for feature in select_features:
  
    fig, axs = plt.subplots(ncols=2, figsize=(20, 8))
    train.groupby([feature, target_feature]).size().unstack().plot(kind='bar', stacked=True, ax=axs[0])
    axs[0].set_xticklabels(axs[0].get_xticklabels(), rotation=90, fontsize=10)
    axs[0].set_ylabel('%')

    train.groupby([feature, target_feature]).size().unstack().apply(lambda x: x/x.sum(), axis=1).plot(kind='bar', stacked=True, ax=axs[1])
    axs[1].set_xticklabels(axs[1].get_xticklabels(), rotation=90, fontsize=10)
    axs[1].set_ylabel('#')
    
    plt.tight_layout()
    plt.show()

In [ ]:
target_feature = train['underlying_conditions_yn']
select_features = [train['hosp_yn'], train['icu_yn'], train['death_yn']]

for feature in select_features:
  
    fig, axs = plt.subplots(ncols=2, figsize=(20, 8))
    train.groupby([feature, target_feature]).size().unstack().plot(kind='bar', stacked=True, ax=axs[0])
    axs[0].set_xticklabels(axs[0].get_xticklabels(), rotation=90, fontsize=10)
    axs[0].set_ylabel('%')

    train.groupby([feature, target_feature]).size().unstack().apply(lambda x: x/x.sum(), axis=1).plot(kind='bar', stacked=True, ax=axs[1])
    axs[1].set_xticklabels(axs[1].get_xticklabels(), rotation=90, fontsize=10)
    axs[1].set_ylabel('#')
    
    plt.tight_layout()
    plt.show()

## Sociological Related Categorical Features Plotted Against Death

Here we are looking discern patterns from our dataset by plotting deaths (yes/no) against a number of sociological features: race, ethnicity, sex.

1. The race of the patient.
2. The ethnicity of the patient.
3. The sex of the patient.

In [ ]:

target_features = [train['race'], train['ethnicity'], train['sex']]
select_features = [train['death_yn']]

for target_feature in target_features:

    for feature in select_features:

        fig, axs = plt.subplots(ncols=2, figsize=(20, 8))
        train.groupby([target_feature, feature]).size().unstack().plot(kind='bar', stacked=True, ax=axs[0])
        axs[0].set_xticklabels(axs[0].get_xticklabels(), rotation=90, fontsize=10)
        axs[0].set_ylabel('%')

        train.groupby([target_feature, feature]).size().unstack().apply(lambda x: x/x.sum(), axis=1).plot(kind='bar', stacked=True, ax=axs[1])
        axs[1].set_xticklabels(axs[1].get_xticklabels(), rotation=90, fontsize=10)
        axs[1].set_ylabel('#')
        
        plt.tight_layout()
        plt.show()

## Observations/Summary:

1. As the COVID19 pandemic progressed, we can generally observe both a reduction in the number of deaths per month and the proporption of deaths with regards the whole, month-on-month. The outlier here is January 2022. From the CDC dataset provided, we cannot discern whether or not the patient was vaccinated, however we can use the months here as a proxy for the Vaccine rollout program in the US, which began mid-December 2020 and had more or less concluded by the end of the following year. The death ratio falling against total cases broadly follows the vaccination rollout scheme, despite this information not formally recorded in our dataset.

2. The data also suggests that age is a predominant and robust factor when gauging the impact contracting COVID will have on your health. Our data shows that those who belong to the oldest age bracket (65+) and test positive for COVID are far more likely to die than those who are younger (compare 65+ against 0-17 years old). Our data also shows that those who are older are more exposed to the risk of COVID, in that the oldest age bucket leads in terms of hospitalisations, ICU admissions, and underlying conditions in comparison to the other age brackets. This aligns with what is now known about COVID-19, that older adults and those with underlying health conditions are considered to be at a higher risk of severe illness and death from COVID-19. 

3. The data also demonstrates that having underlying medical conditions increases the risk of hospitalisation, ICU admission, and death from COVID-19. And while we do not get access to the comorbidities affecting the patients in our dataset, those who reported as having underlying conditions such as diabtes, hypertension, obesity (etc.), confer a much higher risk of death from COVID-19 than others.

4. The data also shows that in the event of being admitted to hospital or the ICU, the success of making a full recovery is limited, and drasticly decreases the older you are. The data also proves the opposite whereby the younger you are the more resilient you are to COVID, leading to far fewer hospitalisation, admissions to ICU, and deaths. 

5. The data also shows that minorities, categorised in our race and ethnicity features (Black, Hispanic, Asian), are hospitalised, admitted to ICU, and die more regularly than their white counterparts pro rata. Without more granular information on our patient it is hard to say exactly what factors are driving these disparities: access to healthcare; wealth; etc.,

6. The data also demonstrates that there is a weak correlation between case_onset_intervals and case_positive_specimen_intervals, with the correlation coefficient is close to zero. This indicates that the timing of when a person experiences symptoms (case onset interval) and when they test positive for COVID-19 (case positive specimen interval) are not strongly associated with each other.

7. Our data also shows that there is a weak negative correlation between our continuous features, and when we map these continuous features against our target feature, there is no real discernible pattern that differents yes from no.


## Selected Features

Based on the above observations, the subset of features we are going to use to model the health profile of our patients are: 

i. <b>Age Group</b>, 
ii. <b>Case Month</b>, 
iii. <b>Hospilisation Status</b>, 
iv. <b>ICU Status</b>, 
v. <b>Underlying conditions</b>. 

Let's transform these features into new features with discrete values.

#### Case Month

In [ ]:
month_map = {'2020-01': 0, 
             '2020-02': 1, 
             '2020-03': 2, 
             '2020-04': 3,
             '2020-05': 4,
             '2020-06': 5,
             '2020-07': 6,
             '2020-08': 7,
             '2020-09': 8,
             '2020-10': 9,
             '2020-11': 10,
             '2020-12': 11,
             '2021-01': 12,
             '2021-02': 13,
             '2021-03': 14,
             '2021-04': 15,
             '2021-05': 16,
             '2021-06': 17,
             '2021-07': 18,
             '2021-08': 19,
             '2021-09': 20,
             '2021-10': 21,
             '2021-11': 22,
             '2021-12': 23,
             '2022-01': 24,
             '2022-02': 25,
             '2022-03': 26,
             '2022-04': 27,
             '2022-05': 28,
             '2022-06': 29,
             '2022-07': 30,
             '2022-08': 31,
             '2022-09': 32,
             '2022-10': 33,
             '2022-11': 34
             }

train['case_month'] = train['case_month'].map(month_map)
test['case_month'] = test['case_month'].map(month_map)

#### Age Group

In [ ]:
age_map = {'0 - 17 years': 0, 
           '18 to 49 years': 1, 
           '50 to 64 years': 2, 
           '65+ years': 3
           }

train['age_group'] = train['age_group'].map(age_map)
test['age_group'] = test['age_group'].map(age_map)

#### Hospitalisation 

In [ ]:
hosp_map = { 'Unknown': 0, 
            'Yes': 1, 
            }

train['hosp_yn'] = train['hosp_yn'].map(hosp_map)
test['hosp_yn'] = test['hosp_yn'].map(hosp_map)

#### ICU 

In [ ]:
icu_map = {'Unknown': 0, 
            'Yes': 1, 
            }

train['icu_yn'] = train['icu_yn'].map(icu_map)
test['icu_yn'] = test['icu_yn'].map(icu_map)

#### Underlying Conditions

In [ ]:
underlying_map = {'Unknown': 0, 
                  'Yes': 1, 
                }

train['underlying_conditions_yn'] = train['underlying_conditions_yn'].map(underlying_map)
test['underlying_conditions_yn'] = test['underlying_conditions_yn'].map(underlying_map)

In [ ]:
# train['definite_underlying'] = (train['underlying_conditions_yn'] == 1).astype(int)
# train['uncertain_underlying'] = (train['underlying_conditions_yn'] == 0).astype(int)

# test['definite_underlying'] = (test['underlying_conditions_yn'] == 1).astype(int)
# test['uncertain_underlying'] = (test['underlying_conditions_yn'] == 0).astype(int)

#### Death

In [ ]:
death_map = {'No': 0,
             'Yes': 1,
             }

train['death_yn'] = train['death_yn'].map(death_map)
test['death_yn'] = test['death_yn'].map(death_map)

# Task 2: Linear Regression

The aim of this task is to perform some basic predictive analysis on the data prepared in task one. To this end, a linear regression model will be created using the training data yielded by the train-test split from the previous task. The features to be analyzed in this section are:

i. <b>age</b>, 
ii. <b>case month</b>, 
iii. <b>hospilisation status</b>, 
iv. <b>ICU status</b>, 
v. <b>underlying conditions</b>. 

 The model will then be evaluated based on its performance using the training data, the test data, and other linear regression models trained on the full dataset. 

## 2.1

In [ ]:
linear_regression_x_train = train[['case_month', 'age_group', 'hosp_yn', 'icu_yn', 'underlying_conditions_yn']]
linear_regression_y_train = train['death_yn']

linear_regression_x_test = test[['case_month', 'age_group', 'hosp_yn', 'icu_yn', 'underlying_conditions_yn']]
linear_regression_y_test = test['death_yn']

In [ ]:
lr = LinearRegression()
lr.fit(linear_regression_x_train, linear_regression_y_train)
linear_regression_y_pred = lr.predict(linear_regression_x_train)

## 2.2

In [ ]:
for feature, coef in zip(linear_regression_x_train.columns, lr.coef_):
    print(feature, ':', coef)

Underlying health conditions: this finding may be counterintuitive, as individuals with underlying health conditions are often thought to be at higher risk for adverse outcomes such as hospitalization or death from COVID-19. However, it is important to remember that the coefficient represents the association between the predictor variable and the target outcome after accounting for the effects of other predictor variables in the model.

In this case, it is possible that other predictor variables, such as age or hospitalization status, may be more strongly associated with the target outcome than underlying health condition, and that the effect of underlying health condition on the target outcome is actually negative when these other factors are taken into account. Therefore, the negative coefficient for "definite_underlying" suggests that the presence of a definite underlying health condition alone may not be a strong predictor of the target outcome in this model.

## 2.3

In [ ]:
# predict target feature values for first 10 training examples
linear_regression_y_train_pred_10 = lr.predict(linear_regression_x_train[:10])
print("Predicted target feature values for first 10 training examples:\n", linear_regression_y_train_pred_10)
print()
# threshold predicted target feature values to get predicted classes
linear_regression_y_train_pred_class_10 = (lr.predict(linear_regression_x_train[:10]) > 0.5).astype(int)
print("Predicted class for first 10 training examples:\n", linear_regression_y_train_pred_class_10)

In [ ]:
# evaluate model on full training set
linear_regression_y_train_pred = lr.predict(linear_regression_x_train)

linear_regression_y_train_pred_class = (linear_regression_y_train_pred > 0.5).astype(int)

linear_regression_accuracy_train = accuracy_score(linear_regression_y_train, linear_regression_y_train_pred_class)
linear_regression_confusion_train = confusion_matrix(linear_regression_y_train, linear_regression_y_train_pred_class)
linear_regression_precision_train = precision_score(linear_regression_y_train, linear_regression_y_train_pred_class)
linear_regression_recall_train = recall_score(linear_regression_y_train, linear_regression_y_train_pred_class)
linear_regression_f1_train = f1_score(linear_regression_y_train, linear_regression_y_train_pred_class)
linear_regression_mse = mean_squared_error(linear_regression_y_train, linear_regression_y_train_pred_class)
linear_regression_r2 = r2_score(linear_regression_y_train, linear_regression_y_train_pred_class)

print("\nEvaluation metrics on the training set:")
print()
print("Accuracy: {:.2f}".format(linear_regression_accuracy_train))
print("Confusion matrix:\n", linear_regression_confusion_train)
print("Precision: {:.2f}".format(linear_regression_precision_train))
print("Recall: {:.2f}".format(linear_regression_recall_train))
print("F1 score: {:.2f}".format(linear_regression_f1_train))
print("MSE: {:.2f}".format(linear_regression_mse))
print("R^2: {:.2f}".format(linear_regression_r2))

In [ ]:
# make predictions on the training set and evaluate the model performance
linear_regression_y_train_pred = lr.predict(linear_regression_x_train)
linear_regression_mae_train = mean_absolute_error(linear_regression_y_train, linear_regression_y_train_pred)
linear_regression_mse_train = mean_squared_error(linear_regression_y_train, linear_regression_y_train_pred)
linear_regression_rmse_train = np.sqrt(linear_regression_mse_train)
linear_regression_r2_train = r2_score(linear_regression_y_train, linear_regression_y_train_pred)

print("Evaluation metrics on the training set:")
print()
print("Mean Absolute Error: {:.2f}".format(linear_regression_mae_train))
print("Mean Squared Error: {:.2f}".format(linear_regression_mse_train))
print("Root Mean Squared Error: {:.2f}".format(linear_regression_rmse_train))
print("R2 score: {:.2f}".format(linear_regression_r2_train))

# make predictions on the test set and evaluate the model performance
linear_regression_y_test_pred = lr.predict(linear_regression_x_test)
linear_regression_mae_test = mean_absolute_error(linear_regression_y_test, linear_regression_y_test_pred)
linear_regression_mse_test = mean_squared_error(linear_regression_y_test, linear_regression_y_test_pred)
linear_regression_rmse_test = np.sqrt(linear_regression_mse_test)
linear_regression_r2_test = r2_score(linear_regression_y_test, linear_regression_y_test_pred)

print("\nEvaluation metrics on the test set:")
print()
print("Mean Absolute Error: {:.2f}".format(linear_regression_mae_test))
print("Mean Squared Error: {:.2f}".format(linear_regression_mse_test))
print("Root Mean Squared Error: {:.2f}".format(linear_regression_rmse_test))
print("R2 score: {:.2f}".format(linear_regression_r2_test))

In [ ]:
print("Classification Report:\n", classification_report(linear_regression_y_train, linear_regression_y_train_pred_class))

# Task 3: Logistic Regression

In [ ]:
logistic_regression_x_train = train[['case_month', 'age_group', 'hosp_yn', 'icu_yn', 'underlying_conditions_yn']]
logistic_regression_y_train = train['death_yn']

logistic_regression_x_test = test[['case_month', 'age_group', 'hosp_yn', 'icu_yn', 'underlying_conditions_yn']]
logistic_regression_y_test = test['death_yn']

In [ ]:
logr = LogisticRegression()
logr.fit(logistic_regression_x_train, logistic_regression_y_train)
logistic_regression_y_pred = logr.predict(logistic_regression_x_train)

In [ ]:
# print coefficients learned by the model
logistic_regression_coef = logr.coef_
logistic_regression_intercept = logr.intercept_
feature_names = logistic_regression_x_train.columns

print("Intercept: {:.2f}".format(logistic_regression_intercept[0]))
print()
for i, name in enumerate(feature_names):
    print("{}: {:.2f}".format(name, logistic_regression_coef[0][i]))


In [ ]:
logistic_regression_y_train_pred_10 = logr.predict_proba(logistic_regression_x_train)[:, 1][:10]
print("Predicted target feature values for the first 10 training examples:\n", logistic_regression_y_train_pred_10)
print()
logistic_regression_y_train_pred_class_10 = (logr.predict(logistic_regression_x_train[:10]) > 0.5).astype(int)
print("Predicted class for the first 10 training examples:\n", logistic_regression_y_train_pred_class_10)

In [ ]:
# predict target feature values for full training set
logistic_regression_y_train_pred = logr.predict_proba(logistic_regression_x_train)[:, 1]

# threshold predicted target feature values at 0.5 to get predicted class for full training set
logistic_regression_y_train_pred_class = (logistic_regression_y_train_pred >= 0.5).astype(int)

logistic_regression_accuracy_train = accuracy_score(logistic_regression_y_train, logistic_regression_y_train_pred_class)
logistic_regression_confusion_train = confusion_matrix(logistic_regression_y_train, logistic_regression_y_train_pred_class)
logistic_regression_precision_train = precision_score(logistic_regression_y_train, logistic_regression_y_train_pred_class)
logistic_regression_recall_train = recall_score(logistic_regression_y_train, logistic_regression_y_train_pred_class)
logistic_regression_f1_train = f1_score(logistic_regression_y_train, logistic_regression_y_train_pred_class)

print("\nEvaluation metrics on the training set:")
print()
print("Accuracy: {:.2f}".format(logistic_regression_accuracy_train))
print("Confusion matrix:\n", logistic_regression_confusion_train)
print("Precision: {:.2f}".format(logistic_regression_precision_train))
print("Recall: {:.2f}".format(logistic_regression_recall_train))
print("F1 score: {:.2f}".format(logistic_regression_f1_train))

In [ ]:
# make predictions on the training set and evaluate the model performance
logistic_regression_y_train_pred = logr.predict(logistic_regression_x_train)
logistic_regression_mae_train = mean_absolute_error(logistic_regression_y_train, logistic_regression_y_train_pred)
logistic_regression_mse_train = mean_squared_error(logistic_regression_y_train, logistic_regression_y_train_pred)
logistic_regression_rmse_train = np.sqrt(logistic_regression_mse_train)
logistic_regression_r2_train = r2_score(logistic_regression_y_train, logistic_regression_y_train_pred)

print("Evaluation metrics on the training set:")
print()
print("Mean Absolute Error: {:.2f}".format(logistic_regression_mae_train))
print("Mean Squared Error: {:.2f}".format(logistic_regression_mse_train))
print("Root Mean Squared Error: {:.2f}".format(logistic_regression_rmse_train))
print("R2 score: {:.2f}".format(logistic_regression_r2_train))

# make predictions on the test set and evaluate the model performance
logistic_regression_y_test_pred = logr.predict(logistic_regression_x_test)
logistic_regression_mae_test = mean_absolute_error(logistic_regression_y_test, logistic_regression_y_test_pred)
logistic_regression_mse_test = mean_squared_error(logistic_regression_y_test, logistic_regression_y_test_pred)
logistic_regression_rmse_test = np.sqrt(logistic_regression_mse_test)
logistic_regression_r2_test = r2_score(logistic_regression_y_test, logistic_regression_y_test_pred)

print("\nEvaluation metrics on the test set:")
print()
print("Mean Absolute Error: {:.2f}".format(logistic_regression_mae_test))
print("Mean Squared Error: {:.2f}".format(logistic_regression_mse_test))
print("Root Mean Squared Error: {:.2f}".format(logistic_regression_rmse_test))
print("R2 score: {:.2f}".format(logistic_regression_r2_test))

In [ ]:
print("Classification Report:\n", classification_report(logistic_regression_y_train, logistic_regression_y_train_pred_class))

# Task 4: Random Forest
### Train a random forest model to predict the target feature

In [ ]:
random_forest_x_train = train[['case_month', 'age_group', 'hosp_yn', 'icu_yn', 'underlying_conditions_yn']]
random_forest_y_train = train['death_yn']

random_forest_x_test = test[['case_month', 'age_group', 'hosp_yn', 'icu_yn', 'underlying_conditions_yn']]
random_forest_y_test = test['death_yn']

rfc = RandomForestClassifier(n_estimators=100, random_state=42)

rfc.fit(random_forest_x_train, random_forest_y_train)

random_forest_y_pred = rfc.predict(random_forest_x_test)

random_forest_accuracy = accuracy_score(random_forest_y_test, random_forest_y_pred)
print("Accuracy: ", random_forest_accuracy)

In [ ]:
feature_importance = pd.DataFrame({'feature': random_forest_x_train.columns, 'importance':rfc.feature_importances_})
feature_importance.sort_values('importance', ascending=False)

In [ ]:
random_forest_y_train_pred_10 = rfc.predict_proba(random_forest_x_train)[:, 1][:10]
print("Predicted target feature values for the first 10 training examples:\n", random_forest_y_train_pred_10)
print()
random_forest_y__train_pred_class_10 = (rfc.predict(random_forest_x_train[:10]) > 0.5).astype(int)
print("Predicted class for the first 10 training examples:\n", random_forest_y__train_pred_class_10)

In [ ]:
# predict target feature values for full training set
random_forest_y_train_pred = rfc.predict_proba(random_forest_x_train)[:, 1]

# threshold predicted target feature values at 0.5 to get predicted class for full training set
random_forest_y_train_pred_class = (random_forest_y_train_pred >= 0.5).astype(int)

random_forest_accuracy_train = accuracy_score(random_forest_y_train, random_forest_y_train_pred_class)
random_forest_confusion_train = confusion_matrix(random_forest_y_train, random_forest_y_train_pred_class)
random_forest_precision_train = precision_score(random_forest_y_train, random_forest_y_train_pred_class)
random_forest_recall_train = recall_score(random_forest_y_train, random_forest_y_train_pred_class)
random_forest_f1_train = f1_score(random_forest_y_train, random_forest_y_train_pred_class)

print("\nEvaluation metrics on the training set:")
print()
print("Accuracy: {:.2f}".format(random_forest_accuracy_train))
print("Confusion matrix:\n", random_forest_confusion_train)
print("Precision: {:.2f}".format(random_forest_precision_train))
print("Recall: {:.2f}".format(random_forest_recall_train))
print("F1 score: {:.2f}".format(random_forest_f1_train))

In [ ]:
print("Classification Report:\n", classification_report(random_forest_y_train, random_forest_y_train_pred_class))

## Random Forest Regressor - Additional Model

In [ ]:
random_forest_reg_x_train = train[['case_month', 'age_group', 'hosp_yn', 'icu_yn', 'underlying_conditions_yn']]
random_forest_reg_y_train = train['death_yn']

random_forest_reg_x_test = test[['case_month', 'age_group', 'hosp_yn', 'icu_yn', 'underlying_conditions_yn']]
random_forest_reg_y_test = test['death_yn']

In [ ]:
rfr = RandomForestRegressor() # create instance of the model
rfr.fit(random_forest_reg_x_train, random_forest_reg_y_train) # fit the model to the training data

random_forest_reg_y_pred = rfr.predict(random_forest_reg_x_test)

#Evaluate the performance of your model:
random_forest_reg_mse = mean_squared_error(random_forest_reg_y_test, random_forest_reg_y_pred)
print("Mean Squared Error:", random_forest_reg_mse)

random_forest_reg_score = rfr.score(random_forest_reg_x_test, random_forest_reg_y_test)
print(f"R^2 score on testing data: {random_forest_reg_score}")

In [ ]:
random_forest_reg_y_train_pred_10 = rfr.predict(random_forest_reg_x_train)[:10]
print("Predicted target feature values for the first 10 training examples:\n", random_forest_reg_y_train_pred_10)
print()
random_forest_reg_y_train_pred_class_10 = (rfr.predict(random_forest_reg_x_train[:10]) > 0.5).astype(int)
print("Predicted class for the first 10 training examples:\n", random_forest_reg_y_train_pred_class_10)

In [ ]:
# predict target feature values for full training set
random_forest_reg_y_train_pred = rfr.predict(random_forest_reg_x_train)

# threshold predicted target feature values at 0.5 to get predicted class for full training set
random_forest_reg_y_train_pred_class = (random_forest_reg_y_train_pred >= 0.5).astype(int)

random_forest_reg_accuracy_train = accuracy_score(random_forest_reg_y_train, random_forest_reg_y_train_pred_class)
random_forest_reg_confusion_train = confusion_matrix(random_forest_reg_y_train, random_forest_reg_y_train_pred_class)
random_forest_reg_precision_train = precision_score(random_forest_reg_y_train, random_forest_reg_y_train_pred_class)
random_forest_reg_recall_train = recall_score(random_forest_reg_y_train, random_forest_reg_y_train_pred_class)
random_forest_reg_f1_train = f1_score(random_forest_reg_y_train, random_forest_reg_y_train_pred_class)

print("\nEvaluation metrics on the training set:")
print()
print("Accuracy: {:.2f}".format(random_forest_reg_accuracy_train))
print("Confusion matrix:\n", random_forest_reg_confusion_train)
print("Precision: {:.2f}".format(random_forest_reg_precision_train))
print("Recall: {:.2f}".format(random_forest_reg_recall_train))
print("F1 score: {:.2f}".format(random_forest_reg_f1_train))

In [ ]:
print("Classification Report:\n", classification_report(random_forest_reg_y_train, random_forest_reg_y_train_pred_class))